### Steps to get similar/duplicate questions

1. Preprocess data - clean question title and body.

remove html, punctuations, stopwords, words less than 4 chars,
but need to keep words like SQL, C++, C#... (get all tags list from tagsDF and exclude these from removing)

2. Apply TF-IDF count vectorizer and create sparse matrix.

3. Use NearestNeighbors and get most similar/duplicate 5 questions.

Challenges - keeping words like SQL, C#, C++ ....

For nearest neighbors, change metrics (mostly need to use cosine metric)

How to relate title and body ??

Can we use title , body and tag together to identify similar questions? Are tags reliable?

In [11]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1045)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1045)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1045)>


In [12]:
questionsDF = pd.read_csv("Questions.csv", encoding='ISO-8859-1')
answersDF = pd.read_csv("Answers.csv", encoding='ISO-8859-1')
tagsDF = pd.read_csv("Tags.csv", encoding='ISO-8859-1')

In [27]:
tagsDF["Tag"].value_counts()

javascript                 124155
java                       115212
c#                         101186
php                         98808
android                     90659
jquery                      78542
python                      64601
html                        58976
c++                         47591
ios                         47009
mysql                       42464
css                         42308
sql                         35782
asp.net                     29970
objective-c                 26922
ruby-on-rails               25789
.net                        24059
c                           23238
iphone                      21539
angularjs                   20345
arrays                      19799
sql-server                  18160
json                        17669
ruby                        17013
r                           15701
ajax                        15629
regex                       15349
xml                         14763
node.js                     14525
asp.net-mvc   

In [35]:
available_tags = tagsDF.Tag.unique()

In [14]:
html_removed_description = questionsDF['Body'].head(10000).apply(lambda x: BeautifulSoup(x,"html.parser").get_text()) 

In [15]:
###Preprocessing logic:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\t", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

def clean_punct(text): 
    text = clean_text(text)
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    #remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in available_tags:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

            
token = ToktokTokenizer()
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'
test=html_removed_description.apply(lambda x: clean_punct(x))           


In [241]:
# tagsDF['Tag'] = tagsDF['Tag'].astype(str)
# grouped_tags = tagsDF.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
# grouped_tags.reset_index()
# grouped_tags_final = pd.DataFrame({'Id':grouped_tags.index, 'Tags':grouped_tags.values})
# #questionsDF.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
# df = questionsDF.merge(grouped_tags_final, on='Id')
# new_df = df[df['Score']>5]
# new_df.drop(columns=['Id', 'Score'], inplace=True)
# new_df['Tags'] = new_df['Tags'].apply(lambda x: x.split())
# # new_df = tagsDF


In [16]:
x_train = test.str.join(' ')
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
x_train = vectorizer.fit_transform(test)


In [17]:
### Store vectorizer object in pickle
import pickle
filename = 'similar_questions_model_vectorizer'
outfile = open(filename,'wb')
pickle.dump(vectorizer,outfile)
outfile.close()

In [19]:
### Read back vectorizer object, to avoid repeatative computation
filename = 'similar_questions_model_vectorizer'
infile = open(filename,'rb')
vect = pickle.load(infile)
infile.close()          

In [ ]:
### Get similarity between questions
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(n_neighbors=5,metric="cosine").fit(x_train)
distances, indices = model_knn.kneighbors(x_train)

In [18]:
# from IPython.display import Audio
# sound_file = "bottle_pop_2.wav"
# Audio(filename=sound_file, autoplay=True)

In [192]:
# question = '<p>I am trying to get my application an installer via WiX 3.0. The exact code is:</p>\n\n<p></p>\n\n<pre><code>&lt;File Id="ServiceComponentMain" Name="$(var.myProgramService.TargetFileName)" Source="$(var.myProgramService.TargetPath)" DiskId="1" Vital="yes"/&gt;\n\n&lt;!-- service will need to be installed under Local Service --&gt;\n&lt;ServiceInstall\n\t\t\t\t\t\tId="MyProgramServiceInstaller"\n\t\t\t\t\t\tType="ownProcess"\n\t\t\t\t\t\tVital="yes"\n\t\t\t\t\t\tName="MyProgramAddon"\n\t\t\t\t\t\tDisplayName="[removed]"\n\t\t\t\t\t\tDescription="[removed]"\n\t\t\t\t\t\tStart="auto"\n\t\t\t\t\t\tAccount="LocalService"\n\t\t\t\t\t\tErrorControl="ignore"\n\t\t\t\t\t\tInteractive="no"/&gt;\n&lt;ServiceControl Id="StartDDService" Name="MyProgramServiceInstaller" Start="install" Wait="no" /&gt;\n&lt;ServiceControl Id="StopDDService" Name="MyProgramServiceInstaller" Stop="both" Wait="yes" Remove="uninstall" /&gt;\n</code></pre>\n\n<p></p>\n\n<p>Thing is, for some reason LocalService fails on the "Installing services" step, and if I change it to "LocalSystem" then the installer times out while trying to start the service.</p>\n\n<p>The service starts fine manually and at system startup, and for all intents and purposes works great. I\'ve heard there are issues getting services to work right under LocalService, but Google isnt really helping as everyone\'s responses have been "got it to work kthx".</p>\n\n<p>Just looking to get this service set up and started during installation, that\'s all. Any help? Thanks!</p>\n'
# question_cleaned = BeautifulSoup(question,"html.parser").get_text()
# question_cleaned = clean_punct(question_cleaned)

In [20]:
def pred_similar(question):
    question_cleaned = BeautifulSoup(question,"html.parser").get_text()
    question_cleaned = clean_punct(question_cleaned)

    testVector = vect.transform(pd.Series(question_cleaned))
    from sklearn.neighbors import NearestNeighbors
    from sklearn.metrics.pairwise import cosine_similarity
    # #model_knn = NearestNeighbors(n_neighbors=5,metric="cosine").fit(question_cleaned)
    # distances, indices = model_knn.kneighbors(testVector)
    #testVector
    sim = cosine_similarity(testVector,x_train)
    
    temp = []
    # now create dictionary adding indices to it
    inds = np.arange(0,10000)
    for row in sim: 
        temp.append(list(zip(inds,row)))

    sortedSims = []
    for row in temp:
        row.sort(key = lambda kv:kv[1], reverse=True)
        sortedSims.append(row)
        
    
    five_similar = []
    for i in range(0,5):
        five_similar.append(test[sortedSims[0][i][0]])
    #similar_question1 = test[sortedSims[0][0][0]]
    
    #return similar_question1
    
    return five_similar

        
    

        

In [33]:
from flask import Flask
from flask import request
from flask import render_template
app = Flask(__name__, template_folder = 'templates')

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    else:
        #tagg = request.form['tag']   
        question = request.form.get('tagg')
        #similar_quest = "Asas"
        similar_quest = pred_similar(question)
        #similar_questions = get_similar()
        return render_template('index.html', similarquestion_1=similar_quest[0], similarquestion_2=similar_quest[1], similarquestion_3=similar_quest[2], similarquestion_4=similar_quest[3], similarquestion_5=similar_quest[4])
   
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Dec/2019 20:48:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2019 20:48:23] "POST / HTTP/1.1" 200 -
